In [2]:
!pip install transformers datasets
!pip install tqdm

In [3]:
from google.colab import files
uploaded = files.upload()

Saving input.txt to input (1).txt


In [7]:
from datasets import Dataset
from transformers import GPT2Tokenizer
import pandas as pd
import io

# Load the text data from the uploaded file
file_content = uploaded['input (1).txt'].decode('utf-8')
df = pd.DataFrame({'text': [file_content]})

# Convert the pandas DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [8]:
from datasets import Dataset

# Step 1: Read raw lines from the file
with open("input.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Step 2: Clean the lines (remove empty ones and strip whitespace)
lines = [line.strip() for line in lines if line.strip() != ""]

# Step 3: Wrap into Hugging Face dataset format
dataset = Dataset.from_dict({"text": lines})

# Preview the first few entries
dataset[:5]


{'text': ['First Citizen:',
  'Before we proceed any further, hear me speak.',
  'All:',
  'Speak, speak.',
  'First Citizen:']}

In [9]:

import os
os.environ["WANDB_DISABLED"] = "true"

In [10]:
def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()  # Needed for training loss
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/32777 [00:00<?, ? examples/s]

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="gpt2-test-run",
    report_to="none",
    num_train_epochs=1,
    max_steps=100,
    per_device_train_batch_size=4,
    save_steps=50,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10
)



In [12]:
small_dataset = tokenized_dataset.select(range(1500))  # use first 1000 samples


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,
)
trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.422700
20,0.391000
30,0.314100
40,0.319200
50,0.326600
60,0.297500
70,0.332100
80,0.316000
90,0.269900
100,0.343700


TrainOutput(global_step=100, training_loss=0.53329021692276, metrics={'train_runtime': 47.4508, 'train_samples_per_second': 8.43, 'train_steps_per_second': 2.107, 'total_flos': 26129203200000.0, 'train_loss': 0.53329021692276, 'epoch': 0.26666666666666666})

In [26]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Hello"
output = generator(prompt, max_length=50, num_return_sequences=1, do_sample=True)

print("📝 Generated Text:\n")
print(output[0]['generated_text'])


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Generated Text:

Hello, how many,
